In [1]:
import os
import json
from django.db import IntegrityError
from django.core.exceptions import ObjectDoesNotExist


from functions.utils import get_data

ROOT_DIR = os.path.dirname(os.path.abspath('__file__'))
DATA_DIR = os.path.join(ROOT_DIR, 'data')

In [2]:
for dirpath, dirnames, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        data = os.path.join(dirpath, filename)
        with open(data) as json_file:
            STORE = json.load(json_file)
                    
        
        # store 생성             
        try:
            store = Store.objects.get(url=STORE['url'])
        except ObjectDoesNotExist:
            store = Store.objects.create(
                url = get_data(STORE, ['url']),
                description = get_data(STORE, ['description']),
                name = get_data(STORE, ['name']),
                store_img = get_data(STORE, ['store_img']),
                is_delivery = get_data(STORE, ['is_delivery']),
                delivery_fee = get_data(STORE, ['delivery']),
                address = get_data(STORE, ['address']),
                lat = get_data(STORE, ['lat']),
                lng = get_data(STORE, ['lng']),
                food_type = get_data(STORE, ['store_type']),
                city = get_data(STORE, ['city'])
            )
            if get_data(STORE, ['is_pickup']): store.is_pickup = STORE['is_pickup']
            if get_data(STORE, ['is_partner']): store.is_partner = STORE['is_partner']
            if get_data(STORE, ['last_order_time']):store.last_order_time = STORE['last_order_time']
            if get_data(STORE, ['estimated_prep_time']) : store.estimated_prep_time = STORE['estimated_prep_time']
            if get_data(STORE, ['delivery_message']) : store.delivery_message = STORE['delivery_message']
        
        # badge 생성
        if STORE['badges']:
            for badge in STORE['badges']:
                try:
                    Badge.objects.create(text=badge['text'])
                except IntegrityError:
                    pass
                finally:
                    B = Badge.objects.get(text=badge['text'])
                    store.badges.add(B)
                
        # open hours 생성
        if STORE['open_hours']:
            for open_hour in STORE['open_hours']:
                try:
                    oh = OpenHour.objects.create(store=store)
                except IntegrityError:
                    oh = OpenHour.objects.get(store=store)
                finally:
                    hour = Hour.objects.create(
                        open_hour = oh,
                        start = open_hour['start_time'],
                        end = open_hour['end_time'],
                        day = open_hour['day_of_week']
                    )
                    oh.hours.add(hour)

        for menu_category in get_data(STORE,['all_menus']):
            mc = MenuCategory.objects.create(
                store=store, 
                name=get_data(menu_category, ['category'])
            )
            
            for menu in get_data(menu_category, ['menus']):
                # menu 생성
                M = Menu.objects.create(
                    name = get_data(menu, ['name']),
                    description = menu['description'].strip() if get_data(menu, ['description']) else None,
                    image_url = get_data(menu, ['img_url']),
                    price = get_data(menu, ['base_price'])
                )
                for option_category in get_data(menu, ['option_groups']):  
                    # option category 생성
                    try:
                        opt_cate = OptionCategory.objects.create(
                            name = get_data(option_category, ['category']),
                            is_required = get_data(option_category, ['is_required'])
                        )
                    except IntegrityError:
                        continue

                    # option 생성
                    for option in get_data(option_category, ['options']):
                        Option.objects.create(
                            category = opt_cate, 
                            name = get_data(option, ['name']), 
                            price = get_data(option, ['price'])
                        )
                    # menu에 option category 추가
                    M.option_categories.add(opt_cate)
                # menu category에 menu 추가
                mc.menu.add(M)
            